In [126]:
import numpy as np
import geopandas
from shapely.geometry import Point, LineString
from shapely.geometry.polygon import Polygon
from math import atan, radians, pi

def find_intersection(point: Point, heading: float, poly: Polygon) -> Point:
    """
    cast a ray from a point with a given heading and check where it intersects the polygon

    ** heading should be in radians! convert it with radians(degrees) if needed
    """

    dy = 1000 # this is an arbitrary distance. 1km should be plenty for our application
    dx = atan(heading) * dy

    p1 = np.array([tmp[0] for tmp in point.coords.xy])
    p2 = p1 + np.array([dx, dy])

    ray = LineString([Point(p1), Point(p2)])

    if not ray.intersects(poly):
        intersection = None
    else:
        xs, ys = ray.intersection(poly).coords.xy
        intersection = Point(list(zip(xs, ys))[1])

    return intersection


def border_dist(point: Point, heading: float, poly: Polygon) -> float:
    """
    calculate distance from point to edge of polygon on some heading 

    ** heading should be in radians! convert it with radians(degrees) if needed
    """

    intersection = find_intersection(point, heading, poly)

    if intersection is None:
        return None
    
    return point.distance(intersection)

In [137]:

# open geojson as UTM zone 10T (lat/ lon in meters)
gp_frame = geopandas.read_file("./export.json").to_crs('EPSG:32610')
border_poly = gp_frame[gp_frame['name'] == 'Survey Area']['geometry'][0]
xx, yy = border_poly.exterior.coords.xy
border_vertices = np.array(list(zip(list(xx), list(yy))))

# here are some test points, also using UTM zone 10T CRS
inside_wp = Point(475968, 4934705)      # lies within polygon
outside_wp = Point(475982, 4934713)     # lies outside of the polygon

# let's check if these points are inside or outside the polygon
assert inside_wp.within(border_poly) is True
assert outside_wp.within(border_poly) is False

# check distances
assert 9 < border_dist(inside_wp, pi, border_poly) < 10
assert border_dist(outside_wp, pi, border_poly) is None